Jupyter notebook tutorial: https://www.youtube.com/watch?v=HW29067qVWk&t=265s
Turicreate hand tracking tutorial: https://martinmitrevski.com/2019/01/19/hand-tracking-with-turi-create-and-core-ml/

In [ ]:
# Confirm that you have CUDA 10
!nvcc --version

In [ ]:
# Install libraries - you might need to restart the runtime after doing this
!pip install turicreate==5.4
# The wrong version of mxnet will be installed
!pip uninstall -y mxnet
# Install CUDA10-compatible version of mxnet
!pip install mxnet-cu100
# install Skafos python sdk
!pip install skafos

In [ ]:
# Import libraries and tell Turicreate to use all GPUs available - this may throw a warning
import urllib
import tarfile
import coremltools
import turicreate as tc
tc.config.set_num_gpus(-1)
import os
from os import listdir
from os.path import isfile, join
import json

In [ ]:

imagesDir = 'pushup-images'

#files = [f for f in listdir(path) if isfile(join(path, f))]

input_file = open ('annotations-sorted.json')
json_array = json.load(input_file)
annotations = []

for item in json_array:
    
    image = item['image']
    annotationItem = item['annotations']
    
#     print(annotation)
    
    label = annotationItem[0]['label']
    coordinates = annotationItem[0]['coordinates']
    
    x = coordinates['x']
    y = coordinates['y']
    width = coordinates['width']
    height = coordinates['height']
    
#     print("label: " + label + "width: " + str(width))
    
    entries = []
    
    coordinates = {'height': height, 'width': width, 'x': x, 'y': y}
    entry = { 'coordinates' : coordinates, 'label' : label }
    annotations.append(entry)
    



In [ ]:
# print('[%s]' % ', '.join(map(str, annotations)))

sf_images = tc.image_analysis.load_images(imagesDir, random_order=False, with_path=True)
sf_images["annotations"] = annotations
sf_images['image_with_ground_truth'] = \
    tc.object_detector.util.draw_bounding_boxes(sf_images['image'], sf_images['annotations'])
sf_images.save('pushupsFrame.sframe')

In [ ]:
#preview sframe

sf_images.explore()

In [ ]:
# Load the data
data =  tc.SFrame('pushupsFrame.sframe')
 
# Make a train-test split
train_data, test_data = data.random_split(0.8)
 
# Create a model
model = tc.object_detector.create(train_data, feature='image', max_iterations=120)
 
# Save predictions to an SArray
predictions = model.predict(test_data)
 
# Evaluate the model and save the results into a dictionary
metrics = model.evaluate(test_data)

In [ ]:
# Export for use in Core ML
model.export_coreml('pushupsTC.mlmodel')